In [54]:
import pandas as pd, docx2txt
from io import StringIO
import os
import re

In [55]:
#read .docx into jupyter
text = docx2txt.process ("C://Users/Bryant/Documents/GitHub/ACT/archive/DataPrep/ACT Math Sheets_v2_3_9_18/Inequalities.docx")

In [56]:
text

'Inequalities\n\n\n\nJune 2015\n\n2. Which of the following inequalities orders the numbers 0.2, 0.03, and ¼ from least to greatest?\n\n0.2 < 0.03 < ¼\n\n0.03 < 0.2 < ¼\n\n0.03 < ¼ < 0.2\n\n¼ < 0.03 < 0.2\n\n¼ < 0.2 < 0.03\n\n23. Given that x ≤ 4 and x + y ≥  5, what is the LEAST value that y can have?\n\n-9\n\n-1 \n\n0\n\n1\n\n9\n\n29. Which of the following inequalities is equivalent to -2x - 6y > 2y - 4?\n\nx < -4y + 2\n\nx > -4y + 2\n\nx < 2y + 2\n\nx < 4y + 2\n\nx > 4y + 2\n\n55. The shaded portion of the graph in the standard (x, y) coordinate plane below represents the solution set of one of the following systems of linear inequalities. Which one?\n\ny ≤ (⅔)x - 6 and y ≥  (-3/2)x + 6\n\ny ≥ (⅔)x - 6 and y ≥  (-3/2)x + 6\n\ny ≥ (⅔)x - 6 and y ≤  (-3/2)x + 6\n\ny ≤ (3/2)x - 6 and y ≥  (-2/3)x + 6\n\ny ≥ (3/2)x - 6 and y ≤  (-2/3)x + 6\n\n*****picture****\n\nDecember 2015\n\n52. One of the following inequalities is graphed below in the standard (x,y) coordinate plane. Which one?\n\

## Clean text. Each question will need the following categories:

#### - ID (to be used as a Primary Key in MySQL db)
#### - Test Date
#### - Question Number
#### - Question Statement
#### - Answer Choices (one choice for each column, 5 total)
#### - Picture (if necessary)

In [57]:
#split text into list separated by \n\n i.e. 'enter' button
text_split = [line for line in StringIO(text).read().split('\n\n')]

In [58]:
#remove empty entries and spaces
try:
    text_split.remove('')
    text_split.remove(' ')
except:
    pass

In [59]:
text_split

['Inequalities',
 'June 2015',
 '2. Which of the following inequalities orders the numbers 0.2, 0.03, and ¼ from least to greatest?',
 '0.2 < 0.03 < ¼',
 '0.03 < 0.2 < ¼',
 '0.03 < ¼ < 0.2',
 '¼ < 0.03 < 0.2',
 '¼ < 0.2 < 0.03',
 '23. Given that x ≤ 4 and x + y ≥  5, what is the LEAST value that y can have?',
 '-9',
 '-1 ',
 '0',
 '1',
 '9',
 '29. Which of the following inequalities is equivalent to -2x - 6y > 2y - 4?',
 'x < -4y + 2',
 'x > -4y + 2',
 'x < 2y + 2',
 'x < 4y + 2',
 'x > 4y + 2',
 '55. The shaded portion of the graph in the standard (x, y) coordinate plane below represents the solution set of one of the following systems of linear inequalities. Which one?',
 'y ≤ (⅔)x - 6 and y ≥  (-3/2)x + 6',
 'y ≥ (⅔)x - 6 and y ≥  (-3/2)x + 6',
 'y ≥ (⅔)x - 6 and y ≤  (-3/2)x + 6',
 'y ≤ (3/2)x - 6 and y ≥  (-2/3)x + 6',
 'y ≥ (3/2)x - 6 and y ≤  (-2/3)x + 6',
 '*****picture****',
 'December 2015',
 '52. One of the following inequalities is graphed below in the standard (x,y) coordi

In [60]:
#store topic
topic = text_split[0]
full_list = []
new = []
for line in text_split[1:]:
    #store month and year of test date
    if re.search('(^December|June|April|July)', line):
        month = re.search('(^December|June|April|July)', line).group()
        year = re.search('$2010|2011|2012|2013|2014|2015|2016|2017|2018|2019|2020',line).group()
        #continue to next iteration if test date is found
        continue
        
    #look for question number to initiate a new row entry
    #len(line) > 10 to avoid finding answers that begin with numbers
    if re.search('^..\.', line) and len(line) > 10:
        #append existing list to full_list, then reinitiate 'new' row
        if len(new) > 0:
            full_list.append(new)
            new = []
        #append various data for 'new' row entry 
        num = re.search('^..\.', line).group().replace(".","")
        #id column
        new.append(month[0:3] + "." + year + "." + num)
        new.append(topic)
        new.append(month + "." + year)
        new.append(num)
        #question statement
        new.append(line)
        continue

    #same as previous block with exception re.search looking for single digits
    if re.search('^.\.', line) and len(line) > 10:
        if len(new) > 0:
            full_list.append(new)
            new = []
        #store question  number    
        num = re.search('^.\.', line).group().replace(".","")
        new.append(month[0:3] + "." + year + "." + num)
        new.append(topic)
        new.append(month + "." + year)
        new.append(num)
        new.append(line)
        continue
    #only lines that reach 'else' statement should be answer choices; append to list   
    else:
        new.append(line)

In [61]:
full_list

[['Jun.2015.2',
  'Inequalities',
  'June.2015',
  '2',
  '2. Which of the following inequalities orders the numbers 0.2, 0.03, and ¼ from least to greatest?'],
 ['Jun.2015.0', 'Inequalities', 'June.2015', '0', '0.2 < 0.03 < ¼'],
 ['Jun.2015.0', 'Inequalities', 'June.2015', '0', '0.03 < 0.2 < ¼'],
 ['Jun.2015.0',
  'Inequalities',
  'June.2015',
  '0',
  '0.03 < ¼ < 0.2',
  '¼ < 0.03 < 0.2',
  '¼ < 0.2 < 0.03'],
 ['Jun.2015.23',
  'Inequalities',
  'June.2015',
  '23',
  '23. Given that x ≤ 4 and x + y ≥  5, what is the LEAST value that y can have?',
  '-9',
  '-1 ',
  '0',
  '1',
  '9'],
 ['Jun.2015.29',
  'Inequalities',
  'June.2015',
  '29',
  '29. Which of the following inequalities is equivalent to -2x - 6y > 2y - 4?',
  'x < -4y + 2',
  'x > -4y + 2',
  'x < 2y + 2',
  'x < 4y + 2',
  'x > 4y + 2'],
 ['Jun.2015.55',
  'Inequalities',
  'June.2015',
  '55',
  '55. The shaded portion of the graph in the standard (x, y) coordinate plane below represents the solution set of one of t

## ready file for import into MySQL db

In [62]:
#prep for import into MySQL db

from pandas import DataFrame
#convert list to df
df = DataFrame.from_records(full_list)
#export to .csv excluding index and header
df.to_csv('Inequalities.csv', index=False, header=False)